In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head(10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,midvagur,62.0511,-7.1939,3.46,75,75,2.06,FO,1704575011
1,1,kinablangan,7.6947,126.5503,25.29,87,99,2.34,PH,1704575012
2,2,ciudad madero,22.2667,-97.8333,24.11,69,75,4.12,MX,1704574980
3,3,grantsville,40.5999,-112.4644,2.08,60,0,4.63,US,1704575013
4,4,dryden,49.7833,-92.7503,-4.84,86,100,1.54,CA,1704575013
5,5,codrington,-38.2667,141.9667,20.29,95,100,3.08,AU,1704575013
6,6,howard springs,-12.4970,131.0470,26.95,95,91,1.22,AU,1704575014
7,7,blackmans bay,-43.0167,147.3167,18.07,79,99,1.34,AU,1704575014
8,8,siwa oasis,29.2041,25.5195,14.99,25,0,3.78,EG,1704575014
9,9,stanley,54.8680,-1.6985,2.66,96,40,2.57,GB,1704575015


In [13]:
%%capture --no-display

# Configure the map plot
Humidity_map=city_data_df.hvplot.points(x='Lng',y="Lat",color='City',marker='o',title='City Humidity',geo=True,tiles="OSM")

# Display the map
Humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)

In [14]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = (city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 21) & (city_data_df['Cloudiness'] == 0) & (city_data_df['Wind Speed'] < 4.5)

ideal_cities=city_data_df[ideal_weather]
# Drop any rows with null values
ideal_cities=ideal_cities.dropna()

# Display sample data
ideal_cities.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
54,54,wailua homesteads,22.0669,-159.3780,26.24,75,0,1.54,US,1704575026
107,107,lihue,21.9789,-159.3672,26.58,74,0,1.54,US,1704575036
205,205,tionk essil,12.7856,-16.5217,25.03,45,0,2.61,SN,1704575068
233,233,holualoa,19.6228,-155.9522,25.31,70,0,4.16,US,1704574847
390,390,sur,22.5667,59.5289,21.53,67,0,2.75,OM,1704575113


In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=ideal_cities[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
54,wailua homesteads,US,22.0669,-159.3780,75,
107,lihue,US,21.9789,-159.3672,74,
205,tionk essil,SN,12.7856,-16.5217,45,
233,holualoa,US,19.6228,-155.9522,70,
390,sur,OM,22.5667,59.5289,67,


In [17]:
# Set parameters to search for a hotel
radius = 10000
params = {"apikeys":"geoapify_key","radius":radius,'type':'accomodation'}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat=row['Lat']
    lng=row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'point({lng},{lat})'
    params["bias"] = f'point({lng},{lat})'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address =response.json()
    
   

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
wailua homesteads - nearest hotel: No hotel found
lihue - nearest hotel: No hotel found
tionk essil - nearest hotel: No hotel found
holualoa - nearest hotel: No hotel found
sur - nearest hotel: No hotel found
taoudenni - nearest hotel: No hotel found
maun - nearest hotel: No hotel found
el colomo - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
54,wailua homesteads,US,22.0669,-159.3780,75,No hotel found
107,lihue,US,21.9789,-159.3672,74,No hotel found
205,tionk essil,SN,12.7856,-16.5217,45,No hotel found
233,holualoa,US,19.6228,-155.9522,70,No hotel found
390,sur,OM,22.5667,59.5289,67,No hotel found
403,taoudenni,ML,22.6783,-3.9836,19,No hotel found
505,maun,BW,-19.9833,23.4167,50,No hotel found
578,el colomo,MX,19.0500,-104.2500,55,No hotel found


In [18]:
# Configure the map plot
map_plot=hotel_df.hvplot.points(x="Lng",y="Lat",hover_cols=["City",'Hotel Name'],size=20,color='City',frame_width=800,frame_height=500,tiles="OSM")

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name)